# Exploratory Data Analysis of User Journeys Across Domains. 
- Exploring user journeys where user sessions travel across the `www.gov.uk` domain to 
  `account.gov.uk` and/or `signin.account.gov.uk`.

- Requirements: Google BigQuery credentials 

### 1.0 Import packages and authentication

In [ ]:
# Imports
import pandas as pd
import seaborn as sns
from google.cloud import bigquery
from google.colab import auth

# Authenticate the user - follow the link and the prompts to get an authentication token
auth.authenticate_user()

### 2.0 Load and prepare data

In [ ]:
query = """
/*
    Retrieves all page hits from sessions that go from govuk to visit at least one cross-domain
    page and then back to govuk, from Google BigQuery.
        - URL parameteres/anchors are removed from the page paths, as we are only
          interested in the the page path.
        - Only page hits are included
        - Print pages are not included
        - Truncate URLs of 'simple_smart_answer', 'local_transaction', 'special_route',
          'licence', 'transaction', and 'Extension' document types, e.g.
          "/claim-tax-refund/y" TO "/claim-tax-refund". This is because we are not
          interested in the user's response, only the page.
    Args:
       - start_date: the start date for the session hit data
       - end_date: the end date for the session hit data
       - seed_hosts: a list of URL slugs that host the cross-domain data

    Returns:
       - A table containing all PAGE hit session data that visits govuk, then
         visits a cross-domain page, then returns to govuk.
*/

DECLARE start_date STRING DEFAULT '20220218';
DECLARE end_date STRING DEFAULT '20220224';

DECLARE seed_hosts ARRAY <STRING>;
SET seed_hosts = ['account.gov.uk', 'signin.account.gov.uk'];

WITH 

primary_data AS (
    SELECT
        hits.hitNumber,
        TIMESTAMP_MILLIS(CAST(hits.time + (visitStartTime * 1000) AS INT64)) AS dateTime,
        REGEXP_REPLACE(hits.page.pagePath, r'[?#].*', '') AS pagePath,
        clientId,
        CONCAT(fullVisitorId, "-", CAST(visitId AS STRING)) AS sessionId,
        hits.page.hostname,
        (SELECT value FROM hits.customDimensions WHERE index = 2) AS documentType
    FROM `govuk-bigquery-analytics.87773428.ga_sessions_*`
    CROSS JOIN UNNEST(hits) AS hits
    WHERE
        _TABLE_SUFFIX BETWEEN start_date AND end_date 
        AND hits.page.pagePath NOT LIKE "/print%"
        AND hits.type = 'PAGE'
    ),

-- truncate URLs of certain document types
sessions_truncate_urls AS (
    SELECT * REPLACE (
        CASE
            WHEN documentType IN ('smart_answer', 'simple_smart_answer', 'local_transaction', 'special_route', 'licence', 'transaction', 'Extension')
            THEN REGEXP_EXTRACT(pagePath, r"^/[^/]+")
            ELSE pagePath
        END AS pagePath
    )
    FROM primary_data
),

-- sessions which visit at least one `seed_hosts`
sessions_with_seed AS (
    SELECT DISTINCT
        sessionId
    FROM sessions_truncate_urls
    WHERE hostname IN UNNEST(seed_hosts)
    GROUP BY sessionId, pagePath
), 

-- all session data (page hits) that visit at least one `seed_hosts`
all_session_data AS (
    SELECT
        clientId,
        sessionId,
        hitNumber,
        dateTime,
        pagePath,
        hostname
    FROM sessions_truncate_urls
    WHERE sessionId IN (SELECT sessionId FROM sessions_with_seed)
),

-- create row number for each session (i.e. create a new hitNumber)
create_row_number AS (
    SELECT
        *,
        ROW_NUMBER() OVER (PARTITION BY sessionId ORDER BY hitNumber) AS rowNumber
    FROM all_session_data
    ORDER BY sessionId, hitNumber, clientId
),

-- get rank for each hostname, group by sessionId
rank_hostnames AS (
    SELECT 
        *,
        DENSE_RANK() OVER (PARTITION BY sessionId ORDER BY hostName DESC) AS rank
    FROM create_row_number
    ORDER BY sessionId, hitNumber, clientId
),

-- only keep ranks where the LAG rank is different or rowNumber= 1
lag_ranks AS (
    SELECT
        *,
        IF((rowNumber = 1) OR (LAG(rank) OVER (PARTITION BY sessionId ORDER BY hitNumber ASC) != rank), TRUE, FALSE) AS preceedingRank
    FROM rank_hostnames
),

-- select distinct hostnames as defined by whether the preceeding rank is different to the current rank 
distinct_hostnames AS (
    SELECT
        *
    FROM lag_ranks
    WHERE preceedingRank
    ORDER BY sessionId, hitNumber, clientId
),

-- count the number of hostnames for each session  
count_hostnames AS (
    SELECT
        *,
        COUNT(hostName) OVER (PARTITION BY sessionId, hostName) AS countHostNamePerSession
    FROM distinct_hostnames
    ORDER BY sessionId, hitNumber, clientId
),

-- select sessions where gov.uk is visited at least twice 
govuk_visits AS (
    SELECT
        * 
    FROM count_hostnames
    WHERE hostName = 'www.gov.uk' 
        AND countHostNamePerSession >= 2
),

-- select sessions which visit a cross domain at least once 
cross_domain_visits AS (
    SELECT
        *
    FROM count_hostnames
    WHERE (hostName = 'signin.account.gov.uk' OR hostName = 'account.gov.uk') 
        AND countHostNamePerSession >= 1
    ORDER BY sessionId, hitNumber, clientId
),

-- only keep sessionIds that are in both govuk_visits and crossdomain_visits
session_ids AS ( 
    SELECT DISTINCT
        govuk_visits.sessionId
    FROM govuk_visits
    INNER JOIN cross_domain_visits 
        ON govuk_visits.sessionId = cross_domain_visits.sessionId
    ORDER BY sessionId
)

-- get all page hit data where sessionIds are in both govuk_visits and cross_domain_visits 
SELECT
    *
FROM create_row_number
WHERE sessionId IN (SELECT sessionId FROM session_ids)
ORDER BY sessionId, hitNumber, clientId
"""

In [ ]:
# Initialise a Google BigQuery client
client = bigquery.Client(project="govuk-bigquery-analytics", location="EU")

# Dry run the query, asking for user input to confirm the query execution size is okay
bytes_processed = client.query(
    query,
    job_config=bigquery.QueryJobConfig(dry_run=True)
).total_bytes_processed

# Compile a message, and flag to the user for a response; if not "yes", terminate execution
user_message = (
    f"This query will process {bytes_processed / (1024 ** 3):.1f} GB when run, "
    + f"which is approximately ${bytes_processed / (1024 ** 4)*5:.3f}. Continue ([yes])? "
)
if input(user_message).lower() != "yes":
    raise RuntimeError("Stopped execution!")

# Execute the query, and return as a pandas.DataFrame
df = client.query(query).to_dataframe()

In [ ]:
df.info()

In [ ]:
df.head()

### 3.0 Research Questions

##### 3.1   How many sessions visit the different domains?

In [ ]:
# Number of unique sessions
df.sessionId.nunique()

In [ ]:
# Number of unique sessions for each hostname 
data = pd.DataFrame(df.groupby('hostname').sessionId.nunique()).reset_index()

ax = sns.barplot(x="hostname", y="sessionId", data=data, hue="hostname", dodge=False)
ax.set_xlabel("Domain")
ax.set_ylabel("Session count")
ax.legend_.remove()
ax.figure.set_size_inches(10,5)

data

##### 3.2   How many sessions visit a page path?

In [ ]:
data = pd.DataFrame(df.groupby(['pagePath', 'hostname']).sessionId.nunique().sort_values(ascending=False)).reset_index()
data.head(20)

In [ ]:
# Distribution of the number of sessionIds that visit a pagePath, i.e. most pagePaths are only visisted during 1 or 2 sessions
ax = sns.histplot(data=data, x=data['sessionId'], bins=120, hue="hostname", element="step")
ax.set_xlim(0, 20)
ax.set_xlabel("sessionId counts")
ax.figure.set_size_inches(10,5)

##### 3.3 What are the unique journeys?

In [ ]:
data = df.groupby(['sessionId'])['pagePath'].apply(', '.join).reset_index()
data = pd.DataFrame(data.groupby(['pagePath']).sessionId.nunique().sort_values(ascending=False)).reset_index()
data

##### 3.3 How many pages in each domain? 

In [ ]:
# Number of pages in each domain 
data = pd.DataFrame(df.groupby('hostname').pagePath.nunique()).reset_index()
data

In [ ]:
# What pages exist in each domain 
data_account = pd.DataFrame(df.where(df['hostname'] == 'account.gov.uk')
                      .groupby(['pagePath'])
                      .sessionId.nunique()
                      .sort_values(ascending=False)
                      .reset_index()
                   )

data_signin = pd.DataFrame(df.where(df['hostname'] == 'signin.account.gov.uk')
                      .groupby(['pagePath'])
                      .sessionId.nunique()
                      .sort_values(ascending=False)
                      .reset_index()
                   )

data_gov = pd.DataFrame(df.where(df['hostname'] == 'www.gov.uk')
                      .groupby(['pagePath'])
                      .sessionId.nunique()
                      .sort_values(ascending=False)
                      .reset_index()
                   )

In [ ]:
ax = sns.histplot(data=data_account, x=data_account['sessionId'], bins=50)
ax.set_xlabel("sessionId counts")
ax.figure.set_size_inches(10,5)

data_account.head(10)

In [ ]:
ax = sns.histplot(data=data_signin, x=data_signin['sessionId'], bins=50)
ax.set_xlabel("sessionId counts")
ax.figure.set_size_inches(10,5)

data_signin.head(10)

In [ ]:
ax = sns.histplot(data=data_gov, x=data_gov['sessionId'], bins=100)
ax.set_xlabel("sessionId counts")
ax.set_xlim(0, 40)
ax.set_ylim(0, 200)

ax.figure.set_size_inches(10,5)

data_gov.head(10)